In [1]:
from pyspark.sql import SparkSession
import pandas as pd

spark = SparkSession.builder.appName('Leetcode').getOrCreate()

24/12/23 12:04:53 WARN Utils: Your hostname, madiv resolves to a loopback address: 127.0.1.1; using 192.168.1.11 instead (on interface wlo1)
24/12/23 12:04:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/23 12:04:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark_df = spark.read.option('delimiter',',').option('header',True).csv('Dataset.csv')
pandas_df = pd.read_csv('Dataset.csv')
spark_df.show()
pandas_df.head()

+----------+--------+----------+
|product_id|low_fats|recyclable|
+----------+--------+----------+
|         0|       Y|         N|
|         1|       Y|         Y|
|         2|       N|         Y|
|         3|       Y|         Y|
|         4|       N|         N|
+----------+--------+----------+



,product_id,low_fats,recyclable
0,0,Y,N
1,1,Y,Y
2,2,N,Y
3,3,Y,Y
4,4,N,N


### SQL Solution

In [3]:
spark_df.createOrReplaceTempView('Products')
sql_solution = spark.sql("SELECT product_id FROM Products WHERE low_fats = 'Y' AND RECYCLABLE = 'Y'")
sql_solution.show()

+----------+
|product_id|
+----------+
|         1|
|         3|
+----------+



### Spark Solution

In [ ]:
spark_solution = spark_df.filter((spark_df['low_fats'] == 'Y') & (spark_df['recyclable'] == 'Y')).select('product_id')
spark_solution.show()

### Pandas Solution

In [ ]:
product_id = pandas_df[(pandas_df['low_fats'] == 'Y') & (pandas_df['recyclable'] == 'Y')].product_id
pandas_solution = pd.DataFrame(product_id)
pandas_solution.head()